# SemEval-2020 Task 4: Commonsense Validation and Explanation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1rlwPIErsvGcYYgM6s_FT9FDn9lcHzvVU?usp=sharing)

# 0. Getting started

In [ ]:
!nvidia-smi

Sun May 16 00:08:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/MyDrive/fer-tar

/content/gdrive/.shortcut-targets-by-id/1drN2cinzIyZ4ThABotRYWgVJfNq9n-5w/fer-tar


In [ ]:
!pip install -q pytorch_lightning
!pip install -q transformers
!pip install -q sentencepiece

     |████████████████████████████████| 808kB 35.4MB/s 
     |████████████████████████████████| 645kB 47.0MB/s 
     |████████████████████████████████| 829kB 55.5MB/s 
     |████████████████████████████████| 276kB 50.5MB/s 
     |████████████████████████████████| 112kB 55.0MB/s 
     |████████████████████████████████| 1.3MB 49.0MB/s 
     |████████████████████████████████| 296kB 54.0MB/s 
     |████████████████████████████████| 143kB 32.4MB/s 
     |████████████████████████████████| 2.3MB 28.3MB/s 
     |████████████████████████████████| 901kB 42.2MB/s 
     |████████████████████████████████| 3.3MB 51.4MB/s 
     |████████████████████████████████| 1.2MB 32.1MB/s 


## 0.1. Imports

In [ ]:
import importlib
import src.models
importlib.reload(src.models)

<module 'src.models' from '/content/gdrive/.shortcut-targets-by-id/1drN2cinzIyZ4ThABotRYWgVJfNq9n-5w/fer-tar/src/models.py'>

In [ ]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from my_scripts import data
from src import encoders
from src.models import RobertaClassifier

In [ ]:
BATCH_SIZE = 32
EPOCHS = 15

In [ ]:
train_ds, val_ds, test_ds = data.ComVEDataset.from_data_folder(x_transforms=[encoders.RobertaEncoder()])
train_dataloader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_ds, batch_size=len(val_ds))

from pytorch_lightning.callbacks.early_stopping import EarlyStopping
early_stop_callback = EarlyStopping(
   monitor='val_acc',
   min_delta=0.00,
   patience=5,
   verbose=False,
   mode='max'
)

model = RobertaClassifier()
trainer = pl.Trainer(max_epochs=EPOCHS, gpus=1, progress_bar_refresh_rate=20, callbacks=[EarlyStopping(monitor='val_loss')], checkpoint_callback=False)
trainer.fit(model, train_dataloader, val_dataloader)

Some weights of the model checkpoint at albert-base-v1 were not used when initializing AlbertModel: ['predictions.LayerNorm.weight', 'predictions.decoder.bias', 'predictions.decoder.weight', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.bias', 'predictions.LayerNorm.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type        | Params
--------------------------------------------
0 | transformer | AlbertModel | 11.7 M
1 | dropo

/content/gdrive/.shortcut-targets-by-id/1drN2cinzIyZ4ThABotRYWgVJfNq9n-5w/fer-tar/src/models.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  acc = torch.tensor((predictions == y_batch).float()).mean()


## 0.2. Getting the data

## 0.3. Preprocessing